## Create a demo database

In [1]:
USE AW_NONE
IF EXISTS (SELECT 1 FROM sys.databases WHERE name = 'CompressGUIDs')
	DROP DATABASE CompressGUIDs;
GO
CREATE DATABASE CompressGUIDs;
GO
USE CompressGUIDs
GO
SET NOCOUNT ON;


--Create two tables, identical except compression levels
--Multiple GUID columns, with all sorts of random values
CREATE TABLE GUIDs (
    GUID1 uniqueidentifier,
    GUID2 uniqueidentifier,
    GUID3 uniqueidentifier,
    CONSTRAINT PK_GUIDs PRIMARY KEY CLUSTERED (GUID1) WITH (DATA_COMPRESSION = NONE)
);
CREATE TABLE GUIDs_PAGE (
    GUID1 uniqueidentifier,
    GUID2 uniqueidentifier,
    GUID3 uniqueidentifier,
    CONSTRAINT PK_GUIDs_PAGE PRIMARY KEY CLUSTERED (GUID1) WITH (DATA_COMPRESSION = PAGE)
);

GO

--insert a bunch of data
--note runtimes will vary

INSERT INTO GUIDs (GUID1, GUID2, GUID3)
SELECT NEWID(), NEWID(), NEWID()
FROM sys.objects o1, sys.objects o2
GO 50

INSERT INTO GUIDs_PAGE (GUID1, GUID2, GUID3)
SELECT NEWID(), NEWID(), NEWID()
FROM sys.objects o1, sys.objects o2
GO 50

: Query failed: Invalid operation. The connection is closed.

Total execution time: 00:00:00.0077176

## Rebuild the indexes

Run statements a bunch of times, and note the runtimes.

In [16]:
--SET STATISTICS IO,TIME ON;
GO

ALTER INDEX PK_GUIDs      ON GUIDs      REBUILD WITH (DATA_COMPRESSION=NONE);
GO 20

ALTER INDEX PK_GUIDs_PAGE ON GUIDS_PAGE REBUILD WITH (DATA_COMPRESSION=PAGE);
GO 20
--SET STATISTICS IO,TIME OFF;

## How does compression affect the index rebuild time/performance?

We're doing more work, so obviously, it takes a little bit longer, right?

### BUT WE SAVED SPACE BECAUSE IT'S COMPRESSED!

How much space did we save?

In [17]:
SELECT DbName             = db_name(database_id) 
      ,TableName          = object_name(object_id,database_id) 
      ,IndexID            = index_id
      ,IndexType          = index_type_desc
      ,AllocUnitType      = alloc_unit_type_desc
      ,IndexSizeKB        = page_count * 8
      ,CompressedPages    = compressed_page_count
      ,UncompressedPages  = page_count - compressed_page_count
      ,AvgRecordSizeBytes = avg_record_size_in_bytes
FROM sys.dm_db_index_physical_stats(db_id('CompressGUIDs'),
                                    object_id('dbo.GUIDs'),
                                    NULL,NULL,'Detailed') -- Have to use Detailed mode to get compression info
WHERE index_level = 0  --Only the leaf level of the index
UNION
SELECT DbName             = db_name(database_id) 
      ,TableName          = object_name(object_id,database_id) 
      ,IndexID            = index_id
      ,IndexType          = index_type_desc
      ,AllocUnitType      = alloc_unit_type_desc
      ,IndexSizeKB        = page_count * 8
      ,CompressedPages    = compressed_page_count
      ,UncompressedPages  = page_count - compressed_page_count
      ,AvgRecordSizeBytes = avg_record_size_in_bytes
FROM sys.dm_db_index_physical_stats(db_id('CompressGUIDs'),
                                    object_id('dbo.GUIDs_PAGE'),
                                    NULL,NULL,'Detailed') -- Have to use Detailed mode to get compression info
WHERE index_level = 0; --Only the leaf level of the index


## Wut?

OK, but sequential GUIDs!

Let's create some sample data with sequential GUIDs


In [19]:
USE CompressGUIDs
GO
SET NOCOUNT ON


--Create two tables, identical except compression levels
--Multiple GUID columns, this time with sequential values.
CREATE TABLE SequentialGUIDs (
    SomeID    tinyint,
    GUID1 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    GUID2 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    GUID3 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    CONSTRAINT PK_SeqGUIDs PRIMARY KEY CLUSTERED (GUID1) WITH (DATA_COMPRESSION = NONE)
);
CREATE TABLE SequentialGUIDs_ROW (
    SomeID    tinyint,
    GUID1 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    GUID2 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    GUID3 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    CONSTRAINT PK_SeqGUIDs_ROW PRIMARY KEY CLUSTERED (GUID1) WITH (DATA_COMPRESSION = ROW)
);
CREATE TABLE SequentialGUIDs_PAGE (
    SomeID    tinyint,
    GUID1 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    GUID2 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    GUID3 uniqueidentifier DEFAULT NEWSEQUENTIALID(),
    CONSTRAINT PK_SeqGUIDs_PAGE PRIMARY KEY CLUSTERED (GUID1) WITH (DATA_COMPRESSION = PAGE)
);
GO

--insert a bunch of data
--note runtimes will vary
INSERT INTO SequentialGUIDs (SomeID)
SELECT 1
FROM AW_NONE.sys.objects o1, AW_NONE.sys.objects o2
GO 20

INSERT INTO SequentialGUIDs_ROW (SomeID)
SELECT 1
FROM AW_NONE.sys.objects o1, AW_NONE.sys.objects o2
GO 20

INSERT INTO SequentialGUIDs_PAGE (SomeID)
SELECT 1
FROM AW_NONE.sys.objects o1, AW_NONE.sys.objects o2
GO 20

Commands completed successfully.

Total execution time: 00:00:00.0006576

: Msg 2714, Level 16, State 6, Line 7
There is already an object named 'SequentialGUIDs' in the database.

Total execution time: 00:00:00.0012384

Beginning execution loop

...

Batch execution completed 20 times...

Total execution time: 00:00:37.2629255

Beginning execution loop

Batch execution completed 20 times...

Total execution time: 00:00:48.4191067